# core

> Audio Embeddings

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
import numpy as np
from fastcore.all import *
import soundfile as sf
from IPython.display import Audio
from fasttransform import Transform, Pipeline
from transformers import AutoFeatureExtractor

/Users/clepelaars/miniconda3/envs/py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class AudioArray(BasicRepr): 
    def __init__(self, a: np.array, sr: int): store_attr()
    @property
    def shape(self): return self.a.shape
    def __len__(self): return len(self.a)
    def __getitem__(self, idx): return self.a[idx]

In [ ]:
aa = AudioArray(np.array([1,2,3,4,5]), 32_000)
aa[:3], aa.sr

(array([1, 2, 3]), 32000)

In [ ]:
def load_audio(path, sr=None): 
    with sf.SoundFile(path) as f: return AudioArray(f.read(), sr if sr else f.samplerate)

In [ ]:
audio = load_audio('../test_files/XC119042.ogg', sr=32_000)
audio[:5], audio.sr

(array([-2.64216160e-05, -2.54259703e-05,  5.56615578e-06, -5.17481631e-08,
        -1.35020821e-06]),
 32000)

In [ ]:
Audio(audio, rate=audio.sr)

In [ ]:
def calc_ratio(audio: AudioArray, target_sr: int) -> float:
    return target_sr / audio.sr

# 16000 / 32000 = 0.5
calc_ratio(audio, 16_000)

0.5

In [ ]:
len(audio.a)

632790

In [ ]:
def new_length(audio: AudioArray, target_sr: int) -> int:
    return int(len(audio.a) * calc_ratio(audio, target_sr))

In [ ]:
# 632790 * 0.5 = 316395
new_length(audio, 16_000)

316395

In [ ]:
def resample(audio: AudioArray, target_sr: int) -> AudioArray:
    if audio.sr == target_sr: return audio
    indices = np.linspace(0, len(audio.a) - 1, new_length(audio, target_sr))
    resampled = np.interp(indices, np.arange(len(audio.a)), audio.a)
    return AudioArray(resampled, target_sr)

In [ ]:
audio_16k = resample(audio, 16_000)
audio_16k[:5], audio_16k.sr

(array([-2.64216160e-05,  5.56613802e-06, -1.35020873e-06,  3.97246192e-06,
         2.80902310e-05]),
 16000)

In [ ]:
Audio(audio_16k, rate=audio_16k.sr)

In [ ]:
def get_features(audio: AudioArray, model_name="MIT/ast-finetuned-audioset-10-10-0.4593"):
    return AutoFeatureExtractor.from_pretrained(model_name)(audio, sampling_rate=audio.sr, return_tensors="np")['input_values']

In [ ]:
inputs = get_features(audio_16k)
inputs.shape, inputs[0][0][:5]

/Users/clepelaars/miniconda3/envs/py312/lib/python3.12/site-packages/transformers/audio_utils.py:297: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


((1, 1024, 128),
 array([-0.7693808 , -1.2774696 , -0.90064937, -1.2775939 , -1.0286174 ],
       dtype=float32))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()